In [1]:
source(file.path("..", "R", "init.R"))
init(where = "local")

suppressMessages({
  library(dplyr)
  library(ggplot2)
  library(reticulate)
  library(lhs)
  library(future.apply)
  library(future)
  library(parallel)
})

[1] "Config for local!"


# Problem

In [2]:
source("problem.R")
problem

$num_vars
[1] 17

$names
 [1] "phen_VegTherTimeResp_X3"   "phen_RepTherTimeResp_X3"  
 [3] "phen_VegPhoMod_X1"         "phen_RepPhoMod_X1"        
 [5] "phen_VegetativeTarget"     "phen_EarlyFloweringTarget"
 [7] "phen_EarlyPodDevTarget"    "phen_FractGrainFill"      
 [9] "phen_MidGrainFill"         "phen_EntGrainFill"        
[11] "phen_Maturing"             "phen_Ripening"            
[13] "leaf_RUE"                  "leaf_AreaLargLeaf"        
[15] "leaf_Phyllochron"          "leaf_ExtinctionCoef_Y1"   
[17] "grain_HarvIndex"          

$bounds
$bounds[[1]]
[1] 21 39

$bounds[[2]]
[1] 21 39

$bounds[[3]]
[1] 10.5 14.5

$bounds[[4]]
[1] 10.5 14.5

$bounds[[5]]
[1] 200 600

$bounds[[6]]
[1] 100 300

$bounds[[7]]
[1]  70 210

$bounds[[8]]
[1] 0.035 0.065

$bounds[[9]]
[1] 0.35 0.65

$bounds[[10]]
[1] 250 750

$bounds[[11]]
[1] 35 65

$bounds[[12]]
[1] 35 65

$bounds[[13]]
[1] 0.8 1.6

$bounds[[14]]
[1] 0.004 0.008

$bounds[[15]]
[1] 35 65

$bounds[[16]]
[1] 0.4 0.8

$bounds[[17]]
[1] 0.35 0.65

# Generate samples

In [3]:
# method = "LHS"
# method = "FAST"
method = "SOBOL"
N_SAMPLES = 500L
samples_df <- generate_samples_csv(problem, method, N_SAMPLES)
head(samples_df)

# options(repr.plot.width = 14, repr.plot.height = 10)
# plot_samples_distribution(samples_df)

,id,phen_VegTherTimeResp_X3,phen_RepTherTimeResp_X3,phen_VegPhoMod_X1,phen_RepPhoMod_X1,phen_VegetativeTarget,phen_EarlyFloweringTarget,phen_EarlyPodDevTarget,phen_FractGrainFill,phen_MidGrainFill,phen_EntGrainFill,phen_Maturing,phen_Ripening,leaf_RUE,leaf_AreaLargLeaf,leaf_Phyllochron,leaf_ExtinctionCoef_Y1,grain_HarvIndex
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,37.56221,30.27361,11.10025,10.97571,212.8854,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132
2,2,21.27010,30.27361,11.10025,10.97571,212.8854,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132
3,3,37.56221,36.98794,11.10025,10.97571,212.8854,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132
4,4,37.56221,30.27361,12.47861,10.97571,212.8854,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132
5,5,37.56221,30.27361,11.10025,13.36828,212.8854,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132
6,6,37.56221,30.27361,11.10025,10.97571,513.0590,153.8067,77.48153,0.04816456,0.5352245,319.5797,45.71864,43.94308,0.9011408,0.007964129,44.49378,0.5641633,0.4061132


# Generate sensi folder and apsimx sims

In [4]:
source(file.path("..", "R", "init.R"))
init(where = "local")

# Create sensi folder
timestamp <- format(Sys.time(), "%Y%m%d_%H%M%S")
sensi_tmp_folder <- create_tmp_dir_from_base_folder(paste0("tmp_sensi_", timestamp))
print(sensi_tmp_folder)

# Copy problem to sensi folder
saveRDS(problem, file.path(sensi_tmp_folder, "problem.rds"))

# Save samples to sensi folder
write.csv(samples_df, file.path(sensi_tmp_folder, "samples.csv"), row.names = FALSE)

# Generate apsimx files
generate_apsimx_from_df(
  samples_df = samples_df,
  folder = sensi_tmp_folder,
  # sensit_base_sim_filepath = file.path(getwd(), "Soy-BR-sensit.apsimx"),
  sensit_base_sim_filepath = file.path(BASE_SIMULATIONS_FOLDER, "Soy-BR-fields-MG6.apsimx"),
  # sensit_base_sim_filepath = file.path(BASE_SIMULATIONS_FOLDER, "Soy-BR-fields-MG06BR_replac.apsimx"),
  runs_only_some = TRUE,
  N = 5,
  parallel = TRUE
)
print_stats_of_folder(sensi_tmp_folder)

[1] "Config for local!"


[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426"
[1] "Number of csvs: 0"
[1] "Number of apsimxs: 5"
[1] "Some files:"
[1] "    /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation1.apsimx"
[2] "    /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation2.apsimx"
[3] "    /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation3.apsimx"
[4] "    /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation4.apsimx"
[5] "    /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation5.apsimx"
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simulation1.apsimx"
[2] "/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp/tmp_sensi_20250124_235426/simula

# Run all apsimx from folder

In [5]:
source(file.path("..", "R", "init.R"))
init(where = "local")

run <- run_apsimx_from_folder(
  folder = sensi_tmp_folder,
  force_rerun = TRUE,
  runs_only_some = FALSE,
  csv_or_db = "db",
  # N = 15,
  # ids_to_run = c(1, 20, 10, 11, 16, 15),
  parallel = TRUE,
  dry_run = FALSE
)

[1] "Config for local!"


# Summarize simulations

In [26]:
source(file.path("..", "R", "init.R"))
init(where = "local")

summarized_df <- summarize_harvest_dbs_from_filelist(
  folder = sensi_tmp_folder,
  # ids_to_summarize = c(1, 2),
  runs_only_some = FALSE,
  # N = 5,
  number_of_fields = 25
)
head(summarized_df)

# Save summarized csv
write.csv(summarized_df, file.path(sensi_tmp_folder, "summarized.csv"), row.names = FALSE)

[1] "Config for local!"


,id,field,yield,biomass,maturity
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,1,CR16,1421.380,4130.941,93
2,1,CR17,1557.952,4453.149,97
3,1,CR18,1662.647,4584.338,97
4,1,FW14a,1500.048,4408.667,96
5,1,FW14b,1705.661,4859.256,92
6,1,FW14c,1638.318,4657.871,88


# Extra: Check difference between summarizes

In [ ]:
# summarized_before <- read.csv("16000_summarized.csv") %>% arrange(id, field)
# summarized_run1 <- read.csv("16000_summarize_run1.csv") %>% arrange(id, field)
# summarized_run2 <- read.csv("16000_summarize_run2.csv") %>% arrange(id, field)

# # rbind(
# #     summarized_before %>% mutate(sim = 1),
# #     summarized_run1 %>% mutate(sim = 2),
# #     summarized_run2 %>% mutate(sim = 3)
# #   ) %>% 
# #   select(sim, id, field, everything()) %>%
# #   arrange(id, field) %>%
# #   head(15)


# # Check duplicated ids
# fields <- df_all_summarized$field %>% unique()
# ids_dupli <- lapply(fields, FUN=function(x){
#   ids <- df_all_summarized %>% filter(field == !!x) %>% filter(duplicated(Biomass)) %>% pull(id)
#   ids
# })
# print(ids_dupli)
# length(duplicated(unlist(ids_dupli)))
# print(Reduce(intersect, ids_dupli))